# hw3 - ensembles

## 1 Подготовка данных

Загрузите и предобработайте данные (по своему усмотрению) из hw1

In [842]:
import pandas as pd
df = pd.read_csv('train_features_with_answers.csv')
#df.describe()
df['age'].value_counts()
df['age'] = df['age'].replace(161,16)
df['age'] = df['age'].replace(181,18)
df['age'] = df['age'].replace(151,15)
df['age'] = df['age'].replace(116,16)
df['age'] = df['age'].replace(5,15)
df['age'] = df['age'].replace(8,18)
df['age'] = df['age'].replace(1,16)
df['age']=df['age'].replace(-1,17)
df.info()
nf = df.copy()
y = nf['G3'].to_numpy().astype(float)
X = nf.drop(['G3'], axis=1)
#X.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 454 entries, 0 to 453
Data columns (total 31 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   school      454 non-null    object 
 1   sex         454 non-null    object 
 2   age         426 non-null    float64
 3   address     449 non-null    object 
 4   famsize     454 non-null    object 
 5   Pstatus     454 non-null    object 
 6   Medu        454 non-null    int64  
 7   Fedu        454 non-null    int64  
 8   Mjob        454 non-null    object 
 9   Fjob        454 non-null    object 
 10  reason      454 non-null    object 
 11  guardian    454 non-null    object 
 12  traveltime  454 non-null    int64  
 13  studytime   454 non-null    int64  
 14  failures    454 non-null    int64  
 15  schoolsup   454 non-null    object 
 16  famsup      454 non-null    object 
 17  paid        454 non-null    object 
 18  activities  454 non-null    object 
 19  nursery     454 non-null    o

In [843]:
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
imputer = SimpleImputer(strategy='median')
#imputer = KNNImputer(n_neighbors=2)
X['age']=imputer.fit_transform(X[['age']])
mode = X['address'].mode().values[0]
X['address'].fillna(value=mode, inplace=True)

Остальная обработка, включая кодирование, дальше

## 2 Обоснуйте выбор слабых (базовых) алгоритмов
Нужно выбрать разные алгоритмы, так как итоговый прогноз будет строится на основании их прогнозов, следовательно чем больше разнообразных по природе, тем лучше.
LogisticRegression для простых линейных зависимостей.
SVC - на прошлой домашке была наиболее достоверной вероятно из-за нелинейности.
GaussianNB - работает на вероятностях. Сам по себе даёт худший результат, но с ним MSE уменьшается на 5.
DecisionTreeClassifier - дерево.
GB - минимизация ошибок по квадрату.

In [844]:
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor

In [845]:
"""def get_models():
    models = []
    models.append(('lr', LinearRegression()))
    models.append(('knn', KNeighborsRegressor(n_neighbors=11)))
    models.append(('cart', DecisionTreeRegressor(max_depth=3)))
    models.append(('svm', SVR(kernel='linear', C=1)))
    return models"""

"def get_models():\n    models = []\n    models.append(('lr', LinearRegression()))\n    models.append(('knn', KNeighborsRegressor(n_neighbors=11)))\n    models.append(('cart', DecisionTreeRegressor(max_depth=3)))\n    models.append(('svm', SVR(kernel='linear', C=1)))\n    return models"

In [846]:
scaler = StandardScaler()
X1=X.copy()
t1=['age','absences','studytime', 'failures', 'famrel']
l1=['school','sex', 'Mjob','Fjob','reason','guardian','higher']
for j in t1:
    X1[j] = scaler.fit_transform(X1[[j]])
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
for i in l1:
    transformed_X1 = encoder.fit_transform(X1[[i]])
    encoded_X1 = pd.DataFrame(transformed_X1, columns=encoder.get_feature_names_out([i]))
    X1 = pd.concat([X1, encoded_X1], axis=1)
    X1.drop(i, axis=1, inplace=True)
X1.drop('Pstatus', axis=1, inplace=True)
X1.drop('paid', axis=1, inplace=True)
X1.drop('famsize', axis=1, inplace=True)
X1.drop('address', axis=1, inplace=True)
X1.drop('Medu', axis=1, inplace=True)
X1.drop('Fedu', axis=1, inplace=True)
X1.drop('traveltime', axis=1, inplace=True)
X1.drop('schoolsup', axis=1, inplace=True)
X1.drop('famsup', axis=1, inplace=True)
X1.drop('activities', axis=1, inplace=True)
X1.drop('nursery', axis=1, inplace=True)
X1.drop('internet', axis=1, inplace=True)
X1.drop('romantic', axis=1, inplace=True)
X1.drop('freetime', axis=1, inplace=True)
X1.drop('goout', axis=1, inplace=True)
X1.drop('Dalc', axis=1, inplace=True)
X1.drop('Walc', axis=1, inplace=True)
X1.drop('health', axis=1, inplace=True)
X1=X1.to_numpy().astype(float)

In [847]:
X2=X.copy()
t2=['age','Medu','Fedu','traveltime', 'studytime', 'failures', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health']
l2=['sex','address','famsize']
for j in t2:
    X2[j] = scaler.fit_transform(X2[[j]])
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
for i in l2:
    transformed_X2 = encoder.fit_transform(X2[[i]])
    encoded_X2 = pd.DataFrame(transformed_X2, columns=encoder.get_feature_names_out([i]))
    X2 = pd.concat([X2, encoded_X2], axis=1)
    X2.drop(i, axis=1, inplace=True)
X2.drop('Pstatus', axis=1, inplace=True)
X2.drop('paid', axis=1, inplace=True)
X2.drop('school', axis=1, inplace=True)
X2.drop('Mjob', axis=1, inplace=True)
X2.drop('Fjob', axis=1, inplace=True)
X2.drop('schoolsup', axis=1, inplace=True)
X2.drop('famsup', axis=1, inplace=True)
X2.drop('activities', axis=1, inplace=True)
X2.drop('nursery', axis=1, inplace=True)
X2.drop('internet', axis=1, inplace=True)
X2.drop('romantic', axis=1, inplace=True)
X2.drop('reason', axis=1, inplace=True)
X2.drop('guardian', axis=1, inplace=True)
X2.drop('higher', axis=1, inplace=True)
print(X2)
X2=X2.to_numpy().astype(float)

          age      Medu      Fedu  traveltime  studytime  failures    famrel  \
0    0.176788 -1.333406  0.653495   -0.772553  -1.097453 -0.383276  0.095112   
1    0.176788  0.452335  0.653495    1.806410  -1.097453 -0.383276 -0.958082   
2   -1.461209  0.452335  1.583548   -0.772553   0.135823 -0.383276  1.148306   
3    0.176788  1.345206  0.653495    0.516929   0.135823  1.289872  0.095112   
4    2.633783 -0.440535 -1.206610    0.516929   0.135823 -0.383276 -3.064470   
..        ...       ...       ...         ...        ...       ...       ...   
449 -1.461209  1.345206 -0.276557   -0.772553   2.602377 -0.383276 -0.958082   
450 -1.461209 -0.440535 -0.276557   -0.772553   2.602377 -0.383276  1.148306   
451 -0.642210  1.345206  1.583548   -0.772553  -1.097453 -0.383276  1.148306   
452 -1.461209 -1.333406 -1.206610    0.516929   0.135823 -0.383276  1.148306   
453 -1.461209  1.345206  1.583548   -0.772553  -1.097453 -0.383276  1.148306   

     freetime     goout      Dalc  ... 

In [848]:
X4=X.copy()
l4=['school','sex','address','famsize','Mjob','Fjob','reason','guardian','schoolsup','famsup','activities','higher','nursery','internet','romantic','paid']
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
for i in l4:
    transformed_X4 = encoder.fit_transform(X4[[i]])
    encoded_X4 = pd.DataFrame(transformed_X4, columns=encoder.get_feature_names_out([i]))
    X4 = pd.concat([X4, encoded_X4], axis=1)
    X4.drop(i, axis=1, inplace=True)
X4.drop('Pstatus', axis=1, inplace=True)
X4.drop('age', axis=1, inplace=True)
#X4.drop('paid', axis=1, inplace=True)
X4.drop('Medu', axis=1, inplace=True)
X4.drop('Fedu', axis=1, inplace=True)
X4.drop('traveltime', axis=1, inplace=True)
X4.drop('studytime', axis=1, inplace=True)
X4.drop('failures', axis=1, inplace=True)
X4.drop('famrel', axis=1, inplace=True)
X4.drop('freetime', axis=1, inplace=True)
X4.drop('goout', axis=1, inplace=True)
X4.drop('Dalc', axis=1, inplace=True)
X4.drop('Walc', axis=1, inplace=True)
X4.drop('health', axis=1, inplace=True)
X4.drop('absences', axis=1, inplace=True)
print(X4)
X4=X4.to_numpy().astype(float)

     school_GP  school_MS  sex_A  sex_B  sex_C  sex_D  sex_F  sex_M  \
0          0.0        1.0    0.0    0.0    0.0    0.0    0.0    1.0   
1          1.0        0.0    0.0    0.0    0.0    0.0    0.0    1.0   
2          1.0        0.0    0.0    0.0    0.0    0.0    0.0    1.0   
3          0.0        1.0    0.0    0.0    0.0    0.0    0.0    1.0   
4          1.0        0.0    0.0    0.0    0.0    0.0    1.0    0.0   
..         ...        ...    ...    ...    ...    ...    ...    ...   
449        1.0        0.0    0.0    0.0    0.0    0.0    0.0    1.0   
450        1.0        0.0    0.0    0.0    0.0    0.0    1.0    0.0   
451        1.0        0.0    0.0    0.0    0.0    0.0    0.0    1.0   
452        0.0        1.0    0.0    0.0    0.0    0.0    1.0    0.0   
453        1.0        0.0    0.0    0.0    0.0    0.0    0.0    1.0   

     address_R  address_U  ...  higher_no  higher_yes  nursery_no  \
0          0.0        1.0  ...        1.0         0.0         0.0   
1        

In [849]:
X5=X.copy()
t5=['age','absences','Medu','Fedu','traveltime','studytime','failures','freetime','famrel','goout','Dalc','Walc','health']
for j in t5:
    X5[j] = scaler.fit_transform(X5[[j]])
X5.drop('Pstatus', axis=1, inplace=True)
X5.drop('paid', axis=1, inplace=True)
X5.drop('school', axis=1, inplace=True)
X5.drop('Mjob', axis=1, inplace=True)
X5.drop('Fjob', axis=1, inplace=True)
X5.drop('schoolsup', axis=1, inplace=True)
X5.drop('famsup', axis=1, inplace=True)
X5.drop('activities', axis=1, inplace=True)
X5.drop('nursery', axis=1, inplace=True)
X5.drop('internet', axis=1, inplace=True)
X5.drop('romantic', axis=1, inplace=True)
X5.drop('reason', axis=1, inplace=True)
X5.drop('guardian', axis=1, inplace=True)
X5.drop('higher', axis=1, inplace=True)
X5.drop('sex', axis=1, inplace=True)
X5.drop('address', axis=1, inplace=True)
X5.drop('famsize', axis=1, inplace=True)
print(X5)
X5=X5.to_numpy().astype(float)

          age      Medu      Fedu  traveltime  studytime  failures    famrel  \
0    0.176788 -1.333406  0.653495   -0.772553  -1.097453 -0.383276  0.095112   
1    0.176788  0.452335  0.653495    1.806410  -1.097453 -0.383276 -0.958082   
2   -1.461209  0.452335  1.583548   -0.772553   0.135823 -0.383276  1.148306   
3    0.176788  1.345206  0.653495    0.516929   0.135823  1.289872  0.095112   
4    2.633783 -0.440535 -1.206610    0.516929   0.135823 -0.383276 -3.064470   
..        ...       ...       ...         ...        ...       ...       ...   
449 -1.461209  1.345206 -0.276557   -0.772553   2.602377 -0.383276 -0.958082   
450 -1.461209 -0.440535 -0.276557   -0.772553   2.602377 -0.383276  1.148306   
451 -0.642210  1.345206  1.583548   -0.772553  -1.097453 -0.383276  1.148306   
452 -1.461209 -1.333406 -1.206610    0.516929   0.135823 -0.383276  1.148306   
453 -1.461209  1.345206  1.583548   -0.772553  -1.097453 -0.383276  1.148306   

     freetime     goout      Dalc      

In [850]:
t=['age','absences','Medu','Fedu','traveltime','studytime','failures','freetime','famrel','goout','Dalc','Walc','health']
l = ['school','sex','address','famsize','Mjob','Fjob','reason','guardian','schoolsup','famsup','activities','higher','nursery','internet','romantic']
for j in t:
    X[j] = scaler.fit_transform(X[[j]])
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
l = ['school','sex','address','famsize','Mjob','Fjob','reason','guardian','schoolsup','famsup','activities','higher','nursery','internet','romantic']
for i in l:
    transformed_X = encoder.fit_transform(X[[i]])
    encoded_X = pd.DataFrame(transformed_X, columns=encoder.get_feature_names_out([i]))
    X = pd.concat([X, encoded_X], axis=1)
    X.drop(i, axis=1, inplace=True)
X.drop('Pstatus', axis=1, inplace=True)
X.drop('paid', axis=1, inplace=True)
print(X)
X=X.to_numpy().astype(float)

          age      Medu      Fedu  traveltime  studytime  failures    famrel  \
0    0.176788 -1.333406  0.653495   -0.772553  -1.097453 -0.383276  0.095112   
1    0.176788  0.452335  0.653495    1.806410  -1.097453 -0.383276 -0.958082   
2   -1.461209  0.452335  1.583548   -0.772553   0.135823 -0.383276  1.148306   
3    0.176788  1.345206  0.653495    0.516929   0.135823  1.289872  0.095112   
4    2.633783 -0.440535 -1.206610    0.516929   0.135823 -0.383276 -3.064470   
..        ...       ...       ...         ...        ...       ...       ...   
449 -1.461209  1.345206 -0.276557   -0.772553   2.602377 -0.383276 -0.958082   
450 -1.461209 -0.440535 -0.276557   -0.772553   2.602377 -0.383276  1.148306   
451 -0.642210  1.345206  1.583548   -0.772553  -1.097453 -0.383276  1.148306   
452 -1.461209 -1.333406 -1.206610    0.516929   0.135823 -0.383276  1.148306   
453 -1.461209  1.345206  1.583548   -0.772553  -1.097453 -0.383276  1.148306   

     freetime     goout      Dalc  ... 

Следующее не нужно, я не прочитала про вероятности, поэтому сделала регрессию. Ниже классификация

In [851]:
"""from scipy import stats
from sklearn.metrics import accuracy_score
import numpy as np

def model(model, X_train, y_train, X_val, X_test, y_test):
    model.fit(X_train, y_train)
    valid = model.predict(X_val)
    prediction = model.predict(X_test)
    print(f' MSE: {mean_squared_error(y_test, prediction)}')
    return prediction, valid

#Base models
first =  GradientBoostingRegressor(random_state=42, criterion = 'squared_error')
second = KNeighborsRegressor(n_neighbors=15)
third = DecisionTreeRegressor(max_depth=2)
fourth = SVR(kernel='linear')

s_m=pd.DataFrame(X_train_m).drop([4,17,31], axis=1).to_numpy().astype(float)
s_v=pd.DataFrame(X_val).drop([4,17,31], axis=1).to_numpy().astype(float)
s_t=pd.DataFrame(X_test).drop([4,17,31], axis=1).to_numpy().astype(float)

t_m=pd.DataFrame(X_train_m).drop([6,42,27,46,38], axis=1).to_numpy().astype(float)
t_v=pd.DataFrame(X_val).drop([6,42,27,46,38], axis=1).to_numpy().astype(float)
t_t=pd.DataFrame(X_test).drop([6,42,27,46,38], axis=1).to_numpy().astype(float)

l_m=pd.DataFrame(X_train_m).drop([52,11,54,32,24], axis=1).to_numpy().astype(float)
l_v=pd.DataFrame(X_val).drop([52,11,54,32,24], axis=1).to_numpy().astype(float)
l_t=pd.DataFrame(X_test).drop([52,11,54,32,24], axis=1).to_numpy().astype(float)

p0,v0 = model(first, X_train_m, y_train_m, X_val, X_test, y_test)
p1,v1=model(second, s_m, y_train_m, s_v, s_t, y_test)
p2,v2=model(third, t_m, y_train_m, t_v, t_t, y_test)
p3,v3=model(fourth, l_m, y_train_m, l_v, l_t, y_test)

s=np.isclose(v1, y_val, atol=0.3)  
t=np.isclose(v2, y_val, atol=0.3) 
f=np.isclose(v3, y_val, atol=0.1)

ps=np.isclose(p1, y_test, atol=0.3)  
pt=np.isclose(p2, y_test, atol=0.3) 
pf=np.isclose(p3, y_test, atol=0.3)

df_p=pd.concat([pd.DataFrame(p0), pd.DataFrame(p1),pd.DataFrame(p2),pd.DataFrame(p3)],axis=1)
df_p=df_p.to_numpy().astype(float)
df_v=pd.concat([pd.DataFrame(v0),pd.DataFrame(v1),pd.DataFrame(v2),pd.DataFrame(v3)],axis=1)
df_v=df_v.to_numpy().astype(float)"""

"from scipy import stats\nfrom sklearn.metrics import accuracy_score\nimport numpy as np\n\ndef model(model, X_train, y_train, X_val, X_test, y_test):\n    model.fit(X_train, y_train)\n    valid = model.predict(X_val)\n    prediction = model.predict(X_test)\n    print(f' MSE: {mean_squared_error(y_test, prediction)}')\n    return prediction, valid\n\n#Base models\nfirst =  GradientBoostingRegressor(random_state=42, criterion = 'squared_error')\nsecond = KNeighborsRegressor(n_neighbors=15)\nthird = DecisionTreeRegressor(max_depth=2)\nfourth = SVR(kernel='linear')\n\ns_m=pd.DataFrame(X_train_m).drop([4,17,31], axis=1).to_numpy().astype(float)\ns_v=pd.DataFrame(X_val).drop([4,17,31], axis=1).to_numpy().astype(float)\ns_t=pd.DataFrame(X_test).drop([4,17,31], axis=1).to_numpy().astype(float)\n\nt_m=pd.DataFrame(X_train_m).drop([6,42,27,46,38], axis=1).to_numpy().astype(float)\nt_v=pd.DataFrame(X_val).drop([6,42,27,46,38], axis=1).to_numpy().astype(float)\nt_t=pd.DataFrame(X_test).drop([6,42

In [852]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
import numpy as np

#Base models
first =  GradientBoostingClassifier(max_depth = 2, random_state=42, criterion = 'squared_error', learning_rate=0.01)
second = KNeighborsClassifier(n_neighbors=15, algorithm= 'kd_tree')
third = DecisionTreeClassifier(max_depth=2)
fourth = SVC(kernel='rbf', probability=True, gamma=0.15)
fifth = GaussianNB()
sixth = LogisticRegression(random_state=0,solver='newton-cholesky',penalty='l2',C = 0.15)

In [853]:
def model(model, X_train, y_train, X_val, X_test, y_val):
    model.fit(X_train, y_train)
    valid = model.predict_proba(X_val)
    prediction = model.predict(X_val)
    print(f' MSE: {mean_squared_error(y_val, prediction)}')
    proba = np.argmax(valid, axis=1)
    #proba = (valid_proba >= 0.5).astype(int)
    predi = model.predict_proba(X_test)
    test =  np.argmax(predi, axis=1)
    return proba, test

X_train, X_test, y_train, y_test = train_test_split(X1, y, test_size=0.25, random_state=42)
X_train_m, X_val, y_train_m, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
v0,p0 = model(first, X_train_m, y_train_m, X_val, X_test, y_val)

X_train, X_test, y_train, y_test = train_test_split(X2, y, test_size=0.25, random_state=42)
X_train_m, X_val, y_train_m, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
v1,p1 = model(second, X_train_m, y_train_m, X_val, X_test, y_val)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
X_train_m, X_val, y_train_m, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
v2,p2 = model(third, X_train_m, y_train_m, X_val, X_test, y_val)

X_train, X_test, y_train, y_test = train_test_split(X4, y, test_size=0.25, random_state=42)
X_train_m, X_val, y_train_m, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
v3,p3 = model(fourth, X_train_m, y_train_m, X_val, X_test, y_val)

X_train, X_test, y_train, y_test = train_test_split(X5, y, test_size=0.25, random_state=42)
X_train_m, X_val, y_train_m, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
v4,p4 = model(fifth, X_train_m, y_train_m, X_val, X_test, y_val)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
X_train_m, X_val, y_train_m, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
v5,p5 = model(sixth, X_train_m, y_train_m, X_val, X_test, y_val)

 MSE: 10.191176470588236
 MSE: 9.470588235294118
 MSE: 9.058823529411764
 MSE: 8.75
 MSE: 21.264705882352942
 MSE: 9.676470588235293


## 3 Постройте решение на основе подхода Blending

Правила:
- Нужно использовать вероятности
- Предложите что-то лучше, чем брать среднее от предсказаний моделей (оценивать уверенность алгоритмов, точности и т.д.)
- Заставьте базовые алгоритмы быть некорелированными
- Добавьте рандома (например, стройте ваши алгоритмы на разных выборках, по разному предобрабатывайте данные или применяйте для разных признаков соответствующие алгоритмы ... )
- Проявите смекалку
- Цель: метрика MSE на тесте меньше 10

In [854]:
from sklearn.ensemble import RandomForestRegressor

#blender = LinearRegression()
v=pd.concat([pd.DataFrame(v0),pd.DataFrame(v1),pd.DataFrame(v2),pd.DataFrame(v4),pd.DataFrame(v5)],axis=1)
p=pd.concat([pd.DataFrame(p0),pd.DataFrame(p1),pd.DataFrame(p2),pd.DataFrame(p4),pd.DataFrame(p5)],axis=1)
blender = RandomForestRegressor (max_depth=2, random_state=0, criterion='squared_error', n_estimators=21)
blender.fit(v, y_val)
#blender.fit(v, y_val)
prediction = blender.predict(p)
predictionint = np.round(blender.predict(p)).astype(int)
print(f' MSE: {mean_squared_error(y_test, prediction)}')
print(f' MSE: {mean_squared_error(y_test, predictionint)}')

 MSE: 8.21750118533107
 MSE: 8.078947368421053


## 4 Постройте решение на основе подхода Stacking

Правила:
- Реализуйте пайплайн обучения и предсказания (например, sklearn.pipeline или класс)
- Проведите оптимизацию пайплайна
- Оцените вклад каждого базового алгоритма в итоговое предсказание
- Цель: метрика MSE на тесте меньше 10

In [855]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
X_train_m, X_val, y_train_m, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [856]:
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

In [857]:
pipeline_lr=Pipeline([('scaler', StandardScaler()),
                      ('lr_classifier',LogisticRegression(random_state=0,solver='newton-cholesky',penalty='l2',C = 0.15))])
pipeline_dt=Pipeline([('scaler', StandardScaler()),
                      ('dt_classifier',DecisionTreeClassifier(max_depth=3))])
pipeline_svm = Pipeline([('scaler', StandardScaler()),
                         ('clf', SVC(kernel='rbf', probability=True, C=1.9))])
pipeline_knn=Pipeline([('scaler', StandardScaler()),
                       ('knn_classifier',KNeighborsClassifier(n_neighbors=15))])
pipeline_gb=Pipeline([('scaler', StandardScaler()),
                      ('rf_classifier', GradientBoostingClassifier(max_depth=2, random_state=42, criterion = 'squared_error', learning_rate=0.05))])
pipeline_nb=Pipeline([('scaler', StandardScaler()),
                      ('NB_classifier', GaussianNB())])

In [858]:
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import KFold
import warnings

warnings.filterwarnings('ignore')

r_lr = cross_val_predict(pipeline_lr, X_train, y_train, 
                                    cv=KFold(n_splits=5, shuffle=True, random_state=42), 
                                    method='predict_proba')
r_dt = cross_val_predict(pipeline_dt, X_train, y_train, 
                                    cv=KFold(n_splits=5, shuffle=True, random_state=42), 
                                    method='predict_proba')
r_svm = cross_val_predict(pipeline_svm, X_train, y_train, 
                                    cv=KFold(n_splits=5, shuffle=True, random_state=42), 
                                    method='predict_proba')
r_knn = cross_val_predict(pipeline_knn, X_train, y_train, 
                                    cv=KFold(n_splits=5, shuffle=True, random_state=42), 
                                    method='predict_proba')
r_gb = cross_val_predict(pipeline_gb, X_train, y_train, 
                                    cv=KFold(n_splits=5, shuffle=True, random_state=42), 
                                    method='predict_proba')
r_lnb = cross_val_predict(pipeline_nb, X_train, y_train, 
                                    cv=KFold(n_splits=5, shuffle=True, random_state=42), 
                                    method='predict_proba')

In [859]:
stack = np.hstack([r_lr,r_dt,r_svm,r_knn,r_gb,r_lnb])

In [860]:
pipelines = [pipeline_lr, pipeline_dt, pipeline_svm, pipeline_knn, pipeline_rf, pipeline_nb]
pipe_dict = {0: 'Logistic Regression', 1: 'Decision Tree', 2: 'Support Vector Machine',3:'K Nearest Neighbor', 4: 'Gradient Boost', 5: 'Naive'}

In [861]:
proba_res=[]
mse=[]
for pipe in pipelines:
    pipe.fit(X_train, y_train)
for i,model in enumerate(pipelines):
    prediction = model.predict(X_test)
    proba = model.predict_proba(X_test)
    proba_res.append(proba)
    mse.append(mean_squared_error(y_test, prediction))
    print(f' MSE: {mean_squared_error(y_test, prediction)}')

 MSE: 9.719298245614034
 MSE: 8.964912280701755
 MSE: 9.62280701754386
 MSE: 12.271929824561404
 MSE: 9.68421052631579
 MSE: 29.219298245614034


In [862]:
stack_t=np.hstack([proba_res[0],proba_res[1],
                   proba_res[2],proba_res[3],
                  proba_res[4],proba_res[5]])

In [863]:
stacker = RandomForestRegressor(max_depth=2, random_state=0, criterion='squared_error')
stacker.fit(stack,y_train)
prediction = stacker.predict(stack_t)
print(f' MSE: {mean_squared_error(y_test, prediction)}')

 MSE: 7.793351413519016
